<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/file_browser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

In [ ]:
import os
import geemap

In [ ]:
in_dir = os.path.join(os.path.expanduser('~'), 'Documents')

In [ ]:
tree = geemap.file_browser(in_dir, show_hidden=False)
tree

In [ ]:
def file_browser(in_dir=None, show_hidden=False):
    """Creates a simple file browser and text editor.

    Args:
        in_dir (str, optional): The input directory. Defaults to None, which will use the current working directory.
        show_hidden (bool, optional): Whether to show hidden files/folders. Defaults to False.

    Returns:
        object: An ipywidget.
    """
    import os
    import ipywidgets as widgets
    from ipytree import Tree, Node
    
    if in_dir is None:
        in_dir = os.getcwd()

    if not os.path.exists(in_dir):
        print('The provided directory does not exist.')
        return
    elif not os.path.isdir(in_dir):
        print('The provided path is not a valid directory.')
        return

    if in_dir.endswith('/'):
        in_dir = in_dir[:-1]

    full_widget = widgets.HBox()
    left_widget = widgets.VBox()

    right_widget = widgets.VBox()

    path_widget = widgets.Text()
    path_widget.layout.min_width = '475px'
    save_widget = widgets.Button(
        description='Save', button_style='primary', tooltip='Save edits to file.')
    info_widget = widgets.HBox()
    info_widget.children = [path_widget, save_widget]

    text_widget = widgets.Textarea()
    text_widget.layout.width = '630px'
    text_widget.layout.height = '600px'

    right_widget.children = [info_widget, text_widget]
    full_widget.children = [left_widget]

    search_box = widgets.Text(placeholder='Search files/folders...')
    search_box.layout.width = '310px'
    tree_widget = widgets.Output()
    tree_widget.layout.max_width = '310px'
    tree_widget.overflow = 'auto'

    left_widget.children = [search_box, tree_widget]

    tree = Tree(multiple_selection=False)
    tree_dict = {}

    def on_button_clicked(b):
        content = text_widget.value
        out_file = path_widget.value

        out_dir = os.path.dirname(out_file)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        with open(out_file, 'w') as f:
            f.write(content)

        text_widget.disabled = True
        text_widget.value = 'The content has been saved successfully.'
        save_widget.disabled = True
        path_widget.disabled = True

        if (out_file not in tree_dict.keys()) and (out_dir in tree_dict.keys()):
            node = Node(os.path.basename(out_file))
            tree_dict[out_file] = node
            parent_node = tree_dict[out_dir]
            parent_node.add_node(node)

    save_widget.on_click(on_button_clicked)

    def search_box_callback(text):

        with tree_widget:
            if text.value == '':
                print('Loading...')
                tree_widget.clear_output(wait=True)
                display(tree)
            else:
                tree_widget.clear_output()
                print('Searching...')
                tree_widget.clear_output(wait=True)
                sub_tree = search_api_tree(text.value, tree_dict)
                display(sub_tree)
    search_box.on_submit(search_box_callback)

    def handle_file_click(event):
        if event['new']:
            cur_node = event['owner']
            for key in tree_dict.keys():
                if (cur_node is tree_dict[key]) and (os.path.isfile(key)):
                    try:
                        with open(key) as f:
                            content = f.read()
                            text_widget.value = content
                            text_widget.disabled = False
                            path_widget.value = key
                            path_widget.disabled = False
                            save_widget.disabled = False
                            full_widget.children = [left_widget, right_widget]
                    except Exception as e:
                        path_widget.value = key
                        path_widget.disabled = True
                        save_widget.disabled = True
                        text_widget.disabled = True
                        text_widget.value = 'Failed to open {}.'.format(
                            cur_node.name) + '\n\n' + str(e)
                        return

    def handle_folder_click(event):
        if event['new']:
            full_widget.children = [left_widget]
            text_widget.value = ''

    root_name = in_dir.split('/')[-1]
    root_node = Node(root_name)
    tree_dict[in_dir] = root_node
    tree.add_node(root_node)
    root_node.observe(handle_folder_click, 'selected')

    for root, d_names, f_names in os.walk(in_dir):

        if not show_hidden:
            folders = root.split('/')
            for folder in folders:
                if folder.startswith('.'):
                    continue
            for d_name in d_names:
                if d_name.startswith('.'):
                    d_names.remove(d_name)
            for f_name in f_names:
                if f_name.startswith('.'):
                    f_names.remove(f_name)

        d_names.sort()
        f_names.sort()

        if root not in tree_dict.keys():
            name = root.split('/')[-1]
            dir_name = os.path.dirname(root)
            parent_node = tree_dict[dir_name]
            node = Node(name)
            tree_dict[root] = node
            parent_node.add_node(node)
            node.observe(handle_folder_click, 'selected')

        if len(f_names) > 0:
            parent_node = tree_dict[root]
            parent_node.opened = False
            for f_name in f_names:
                node = Node(f_name)
                node.icon = 'file'
                full_path = os.path.join(root, f_name)
                tree_dict[full_path] = node
                parent_node.add_node(node)
                node.observe(handle_file_click, 'selected')

    with tree_widget:
        tree_widget.clear_output()
        display(tree)

    return full_widget

In [ ]:
tree = file_browser()
tree